In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as plt
import datetime
import os
import glob

# packages burrowed and adapted from pypmi
import sys
sys.path.insert(1, '/scratch/c.c21013066/PPMI_DataPreparation/phenotype/')
import _info2021
import _utils
import _loaders
import _loadersSubitems
import _thresholds2021 as thr

from importlib import reload

In [ ]:
path = "/scratch/c.c21013066/data/ppmi/phenotypes2021"

In [ ]:
demographics = _loaders.load_demographics(path=path)
# check for unreasonable data in demographics
# go through columns and check data
reload(thr)
demographics_clean = demographics.copy(deep=True)

for col in demographics.columns[1:]:
    print(col)
    info = thr.DEMOGRAPHIC_INFO[col]
    print('    Are there NaN values? {}'.format(demographics_clean[col].isna().sum()))
    if info['scale_level'] == 'categorical': # check if only allowed categories used
        if col == 'diagnosis':
            print('    How many dropped due to no category? {}'.format(demographics_clean[col].isna().sum()))
            # we can only use data of people who belong to a category
            demographics_clean = demographics_clean.dropna(axis='rows',how='any',subset=[col])
        assert all(elem in info['categories']  for elem in set(demographics_clean[col].dropna()))
    if info['scale_level'] == 'date': # check if date in range
        # need brithdate information
        if col == 'date_birth':
            print('    How many dropped due to no category? {}'.format(demographics_clean[col].isna().sum()))
            # we can only use data of people who belong to a category
            demographics_clean = demographics_clean.dropna(axis='rows',how='any',subset=[col])
        assert all(demographics_clean[col].dropna() >= info['min']), 'min exceeds bounds {}'.format(demographics_clean[col].min())
        assert all(demographics_clean[col].dropna() <= info['max']), 'max exceeds bounds {}'.format(demographics_clean[col].max())
    if info['scale_level'] == 'binomial':
        assert  set(demographics_clean[col].dropna()) == set(info['categories'])
    if info['scale_level'] == 'normal':
        assert all(demographics_clean[col].dropna() >= info['min']), 'min exceeds bounds {}'.format(demographics_clean[col].min())
        assert all(demographics_clean[col].dropna() <= info['max']), 'max exceeds bounds {}'.format(demographics_clean[col].max())
demographics_clean.to_csv(f'{path}/demographics_clean.csv')

# Digital features

in the scripts folder, run_feature_extraction.sh calls feature_extraction.py which uses tsfresh on the timeseries features provided by Verily. That creates the extracted_features folder.